In [1]:
import numpy as np
import tensorflow as tf
from utils_functions import *
from plan import *
import random
from main import *

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils  import plot_model
from tensorflow.keras.utils  import Sequence
import datetime, os

In [2]:
# controllo di star effettivamente usando la GPU

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))
print("nome gpu: ",tf.test.gpu_device_name())

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
nome gpu:  /device:GPU:0


## Preparazione dati

### Preparazione dati per struttura Simil-CBOW

In [3]:
# window_size = 2
# dizionario_stati = load_file("./dizionario_stati")
# piani_caricati = load_file("./plans")

In [4]:
# def crea_vettori_finestre(dizionario, lista_piani, dimensione_finestra):
#     finestre = []
#     outputs = []
#     print("Qui 000")
#     tutti_vettori_1x340 = costruisci_tutti_vettori_1x340(dizionario, lista_piani)
#     print("Qui 00")
#     tutti_vettori_1x340_list= tutti_vettori_1x340.tolist()
#     print("Qui 0")
    
#     for vett in tutti_vettori_1x340_list:
#         indice = tutti_vettori_1x340_list.index(vett)
#         if indice == 0 or indice == 1 or indice == len(tutti_vettori_1x340_list)-2 or indice == len(tutti_vettori_1x340_list)-1:
#             print("Qui 1", indice)
#             continue  # se non posso prendere 2 vettori prima, o due vettori poi, non lo prendo, non può essere un output
#             print("Qui 2")
#         else:
#             print("Qui 3")
#             start = indice - dimensione_finestra  # 2 prima del corrente
#             end = indice + dimensione_finestra    # 2 dopo il corrente
#             vettore_finestra = []
#             for i in range(start, end):
#                 if i == indice:
#                     print("Qui 4")
#                     outputs.append(tutti_vettori_1x340[i])  # se è il vettore centrale, la Seq0 dell'immagine di riferimento
#                                                  # allora è un autput della finestra, e lo aggiungo al vettore degli output
#                 else:
#                     print("Qui 5")
#                     vettore_finestra.append(
#                         tutti_vettori_1x340[i])  # se è Seq-2, o Seq-1, o Seq+1, o Seq+2, lo aggiungo al vettore finestra
#         finestre.append(vettore_finestra)

#     print("Qui 6")
#     return finestre, outputs

In [5]:
# crea_vettori_finestre(dizionario_stati, piani_caricati, window_size)

Creazione dataset

In [6]:
dizionario_stati = load_file("./dizionario_stati")
piani_caricati = load_file("./plans")

File loaded
File loaded


In [7]:
dataset = costruisci_vettore_per_piano(dizionario_stati,piani_caricati)

In [8]:
#ESEMPIO DI TEST
d = [
    [[0,0,0,0],[0,0,0,1],[0,0,0,2],[0,0,0,3],[0,0,0,4],[0,0,0,5],[0,0,0,6],[0,0,0,7]],
    [[1,1,1,0],[1,1,1,1],[1,1,1,2],[1,1,1,3],[1,1,1,4],[1,1,1,5],[1,1,1,6],[1,1,1,7],[1,1,1,8],[1,1,1,9]],
    [[2,2,2,0],[2,2,2,1],[2,2,2,2],[2,2,2,3],[2,2,2,4]],
    [[3,3,3,0],[3,3,3,1],[3,3,3,2],[3,3,3,3]],
    [[4,4,4,0],[4,4,4,1],[4,4,4,2],[4,4,4,3],[4,4,4,4],[4,4,4,5]]

]

In [9]:
new = dataset_augmentation(d)

In [10]:
print(new)
print(new.shape)

[[[0 0 0 0]
  [0 0 0 1]
  [0 0 0 2]
  [0 0 0 3]
  [0 0 0 4]]

 [[1 1 1 0]
  [1 1 1 1]
  [1 1 1 2]
  [1 1 1 3]
  [1 1 1 4]]

 [[1 1 1 5]
  [1 1 1 6]
  [1 1 1 7]
  [1 1 1 8]
  [1 1 1 9]]

 [[2 2 2 0]
  [2 2 2 1]
  [2 2 2 2]
  [2 2 2 3]
  [2 2 2 4]]

 [[4 4 4 0]
  [4 4 4 1]
  [4 4 4 2]
  [4 4 4 3]
  [4 4 4 4]]]
(5, 5, 4)


In [11]:
dataset_5x340 = dataset_augmentation(dataset)

In [12]:
print(dataset_5x340.shape)

(256153, 5, 340)


Prossimi passi:
* shuffle di dataset_5x340
* splittare in train_x,train_y , val_x,val_y, test_x,test_y
* la parte "x" dim: 4x340, "y" dim: 1x340, con "y" il primo vettore della sequenza --> non il terzo? il prof parlava di dare in pasto seq-2, seq-1, seq+1, seq+1, per ottenere seq0
* comprimerli rendendoli statici

In [71]:
def crea_sets_parte2(dataset_gruppi_da_5):
    random.shuffle(dataset_gruppi_da_5)
    split = int(len(dataset_gruppi_da_5)//3)
    train = dataset_gruppi_da_5[split:]
    val_test = dataset_gruppi_da_5[:split]
    sub_split = int(len(val_test)//2)
    validation = val_test[sub_split:]
    test = val_test[:sub_split]
    
    l_train_x = []
    l_validation_x = []
    l_test_x = []
    
    l_train_y = []
    l_validation_y = []
    l_test_y = []
    
    for t in train: # scorro ogni quintupla
        l_train_y.append(t[2]) # metto l'elemento centrale della quintupla nel set y
        list_t = t.tolist()
        list_t.remove(list_t[2]) # elimino l'elemento centrale dalla quintupla
        l_train_x.append(list_t) # aggiungo la quadrupla al set x
        
    for v in validation: # scorro ogni quintupla
        l_validation_y.append(v[2]) # metto l'elemento centrale della quintupla nel set y
        list_v = v.tolist()
        list_v.remove(list_v[2]) # elimino l'elemento centrale dalla quintupla
        l_validation_x.append(list_v) # aggiungo la quadrupla al set x
        
    for te in test: # scorro ogni quintupla
        l_test_y.append(te[2]) # metto l'elemento centrale della quintupla nel set y
        list_te = te.tolist()
        list_te.remove(list_te[2]) # elimino l'elemento centrale dalla quintupla
        l_test_x.append(list_te) # aggiungo la quadrupla al set x
    
    train_x = np.array(l_train_x)
    validation_x = np.array(l_validation_x)
    test_x = np.array(l_test_x)
    
    train_y = np.array(l_train_y)
    validation_y = np.array(l_validation_y)
    test_y = np.array(l_test_y)
    
    save_file(train_x, "./Dataset_Parte2/", "set_training_x")
    save_file(validation_x, "./Dataset_Parte2/", "set_validation_x")
    save_file(test_x, "./Dataset_Parte2/", "set_test_x")
    
    save_file(train_y, "./Dataset_Parte2/", "set_training_y")
    save_file(validation_y, "./Dataset_Parte2/", "set_validation_y")
    save_file(test_y, "./Dataset_Parte2/", "set_test_y")
    
    return True

In [72]:
crea_sets_parte2(dataset_5x340)

set_training_x saved in ./Dataset_Parte2/
set_validation_x saved in ./Dataset_Parte2/
set_test_x saved in ./Dataset_Parte2/
set_training_y saved in ./Dataset_Parte2/
set_validation_y saved in ./Dataset_Parte2/
set_test_y saved in ./Dataset_Parte2/


True